Initiliaze and prepare all models

In [1]:
# ~~~~~~~~~~~~~~~~
# PyTorch Model

import torch
import yaml
from models.decoder import architectures

with open("models/decoder/decoder_params.yaml") as file:
    params = yaml.safe_load(file)

model = architectures.LightningNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
model.load_state_dict(torch.load("models/decoder/decoder.pth"))
model.eval();

model_denoise = architectures.LightningNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
model_denoise.load_state_dict(torch.load("models/decoder/decoder_denoise.pth"))
model_denoise.eval();

# ~~~~~~~~~~~~~~~~
# Kernel DM+V

from models.kernel_dmv.my_kernel_dmv import KernelDMV
kdm = KernelDMV()

# ~~~~~~~~~~~~~~~~
# GMRF

from models.gmrf.my_gmrf import myGMRF
gmrf = myGMRF()

### Data & Metrics

Specify data

In [2]:
from torch.utils import data
from data.gdm_dataset import GasDataSet

dataset = GasDataSet("data/30x25/test.pt")

Specify metrics

In [3]:
import gdm_metrics

rmse = {"decoder": 0,
        "decoder_denoise": 0,
        "gmrf": 0,
        "kdm": 0}

kld = {"decoder": 0,
       "decoder_denoise": 0,
       "gmrf": 0,
       "kdm": 0}

### Run

In [5]:
from tqdm import tqdm

list_of_models = [
                  #"decoder",
                  #"decoder_denoise",
                  "gmrf",
                  #"kdm",
                 ]

dataloader = iter(data.DataLoader(dataset, shuffle=False, drop_last=True))

for X, y in tqdm(dataloader):
    with torch.no_grad(): 
        if "decoder" in list_of_models:
            y_decoder = model(X.squeeze(1))
            rmse["decoder"] += gdm_metrics.rmse(y_decoder, y)
            kld["decoder"] += gdm_metrics.kld(y_decoder, y)
        
        if "decoder_denoise" in list_of_models:
            y_decoder_denoise = model_denoise(X.squeeze(1))
            rmse["decoder_denoise"] += gdm_metrics.rmse(y_decoder_denoise, y)
            kld["decoder_denoise"] += gdm_metrics.kld(y_decoder_denoise, y)
        
        if "gmrf" in list_of_models:
            y_gmrf = gmrf.calculate(y.squeeze())[None,None,:]  # add two empty dimensions to be consistent with y.shape
            rmse["gmrf"] += gdm_metrics.rmse(y_gmrf, y)
            kld["gmrf"] += gdm_metrics.kld(y_gmrf, y)
            
        if "kdm" in list_of_models:
            y_kdm = kdm.calculate(y.squeeze())[None,None,:]    # add two empty dimensions to be consistent with y.shape
            rmse["kdm"] += gdm_metrics.rmse(y_kdm, y)
            kld["kdm"] += gdm_metrics.kld(y_kdm, y)

100%|█████████████████████████████████████| 32400/32400 [08:47<00:00, 61.46it/s]


### Evaluation

In [6]:
# Print results
print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataloader)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataloader)}")

########
# RMSE #
########
decoder: 	 0.0007184731075540185
decoder_denoise: 	 0.000727061415091157
gmrf: 	 0.11210330681971697
kdm: 	 0.001143007651812626
########
# KL D #
########
decoder: 	 2.8044156351825222e-05
decoder_denoise: 	 2.748418046394363e-05
gmrf: 	 0.008461204471344255
kdm: 	 6.948788059839356e-05


Previous results

In [5]:
# Print results
print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataloader)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataloader)}")

########
# RMSE #
########
decoder: 	 0.07057706266641617
decoder_denoise: 	 0.07306750863790512
gmrf: 	 0.11100773428192476
kdm: 	 0.11176697431124034
########
# KL D #
########
decoder: 	 0.0032490254379808903
decoder_denoise: 	 0.003500421764329076
gmrf: 	 0.008397885626507435
kdm: 	 0.00816277310695924
